### extract_filament
will extract the filament from a the EAGLE simulation.

In [4]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u
from astropy.io import fits

import os

%matplotlib inline

import get_halpha_SB

%run 'load_data.ipynb'

In [ ]:
def plotfilament(SBdata_5,ax):
    clabel = r'log photons/cm$^2$/s/sr'; Vmin = None; Vmax= None
    fontsize=13
    ax.set_xlabel(r'X [cMpc]',fontsize=fontsize)
    ax.set_ylabel(r'Y [cMpc]',fontsize=fontsize)
    
    ax.tick_params(labelsize=fontsize)
    colmap = 'viridis' #'afmhot'
    ax.patch.set_facecolor(cm.get_cmap(colmap)(0.)) # sets background color to lowest color map value

    img = ax.imshow(SBdata_5,origin='lower', cmap=cm.get_cmap(colmap), vmin = Vmin, vmax=Vmax,interpolation='nearest')

    div = axgrid.make_axes_locatable(ax)
    cax = div.append_axes("right",size="15%",pad=0.1)
    cbar = plt.colorbar(img, cax=cax)
    cbar.solids.set_edgecolor("face")
    cbar.ax.set_ylabel(r'%s' % (clabel), fontsize=fontsize)
    cbar.ax.tick_params(labelsize=fontsize)

def extractdata(xfull,yfull,data):
    SBdata = np.zeros(xfull.shape)
    for i in range(yfull.shape[0]):
        for j in range(yfull.shape[1]):
                SBdata[i,j]  = data[xfull[i,j],yfull[i,j]]
    return SBdata

def extractfilament(data_tuple=None,plotchecks=True,snapnum = 28,region=1,machine=None,cutout=None):
    if data_tuple is None and cutout is None:
        if machine is None:
            print 'ERROR: No data_tuple supplied and no machine entered -- returning None.'
            return None
        else:
            print 'No data_tuple supplied, loading data_tuple for res=None,distance=50Mpc,slicewidth=20...'
            data_tuple = loaddata(machine=machine,resolution=None,distance='50Mpc',slicewidth=20.)
    
    fulldata = None; fullsize = None
    if cutout is not None:
        print 'cutout is supplied - will extract filament from the cutout.'
        fulldata = cutout
        fullsize = fulldata.shape[0]
        size = 20.
        xystarts = [40.,0.]
    else:
        fulldata = data_tuple[0]
        fullsize = fulldata.shape[0]
        size = 100.
    
    if plotchecks:
        if cutout is not None:
            plotxystarts = [40.,0.]
            plotsize     = 20.
            fig, (ax) = plt.subplots(1,1,figsize = (10., 10.))
            get_halpha_SB.makemap(fulldata,plotsize,ax,xystarts = plotxystarts)
        else:
            plotxystarts = [40.,0.]
            plotsize     = 20.
            fig, (ax) = plt.subplots(1,1,figsize = (10., 10.))
            get_halpha_SB.makemap(fulldata[(40./100.*fullsize):(60./100.*fullsize),0:(20./100.*fullsize)],\
                                  plotsize,ax,xystarts = plotxystarts)

    if snapnum == 27:
        if plotchecks:
            # Pick out regions along the filaments in the map (snapnum 27)
            ax.plot([53,53,56,56,53],[9.2,10,8.5,7.7,9.2],color='r', label='Region 3')
            ax.plot(np.array([46.2,47.2,48.2,47.2,46.2]),[14,14,10,10,14],color='r', label='Region 1')
            ax.plot(np.array([43,43,46,46,43]),[7.5,8.2,7.2,6.5,7.5],color='r', label = 'Region 2')
    
        xbox_3 = (np.array([53,53,56,56])-xystarts[0])*fullsize/size
        ybox_3 = (np.array([9.2,10,8.5,7.7])-xystarts[1])*fullsize/size

        xbox_2 = (np.array([43,43,46,46])-xystarts[0])*fullsize/size
        ybox_2 = (np.array([7.8,8.,7.1,6.8])-xystarts[1]-0.05)*fullsize/size

        xbox_1 = (np.array([47.4,46.2,46.9,48.1])-xystarts[0])*fullsize/size
        ybox_1 = (np.array([10.5,14,14,10.5])-xystarts[1])*fullsize/size
    

    if snapnum == 28:
        if plotchecks:
            # Pick out regions along the filaments in the map (snapnum 28)
            ax.plot([53,53,56,56,53],np.array([9.2,10,8.5,7.7,9.2])-0.2,color='r', label='Region 3')
            ax.plot(np.array([46.2,47.2,48.2,47.2,46.2])+0.5,[14,14,10,10,14],color='g', label='Region 1')
            ax.plot(np.array([43,43,46,46,43]),np.array([7.5,8.2,7.2,6.5,7.5])+0.2,color='c', label = 'Region 2')
    
        xbox_3 = (np.array([53,53,56,56])-xystarts[0])*fullsize/size
        ybox_3 = (np.array([9.2,10,8.5,7.7])-0.2-xystarts[1])*fullsize/size

        xbox_2 = (np.array([43,43,46,46])-xystarts[0])*fullsize/size
        ybox_2 = (np.array([7.8,8.,7.1,6.8])-0.05+0.2-xystarts[1])*fullsize/size

        xbox_1 = (np.array([47.4,46.2,46.9,48.1])+0.5-xystarts[0])*fullsize/size
        ybox_1 = (np.array([10.5,14,14,10.5])-xystarts[1])*fullsize/size
    
    xbox=None;ybox=None
    if region==1:
        xbox = xbox_1
        ybox = ybox_1
        print('Region 1: '+str(xbox)+' , '+str(ybox))
    elif region==2:
        xbox = xbox_2
        ybox = ybox_2
        print('Region 2: '+str(xbox)+' , '+str(ybox))
    elif region==3:
        xbox = xbox_3
        ybox = ybox_3
        print('Region: '+str(xbox)+' , '+str(ybox))    
    else:
        print "ERROR: Must set region equal to 1 or 2 or 3 -- no other options! Returning None."
        return None
    
    xfull, yfull= get_halpha_SB.indices_region(xbox.astype(int),ybox.astype(int))

    SBdata = extractdata(xfull,yfull,fulldata)

    if plotchecks:
        print "Plotting the data that we've pulled out from the array as a check"
        fig, (ax) = plt.subplots(1,1,figsize = (7.5, 8.))
        plotfilament(SBdata,ax)
    
       # # Plot the original data around the region we pulled out to do a cross-check
       # #fig = plt.figure(figsize = (16.5, 15.))
       # ax1 = plt.subplot(422)
       # size = 4.0
       # xystarts = [45.2,10.]
       # get_halpha_SB.makemap(data_5[(45.2/100.*3200.):(49.2/100.*3200.),(10./100.*3200.):(14./100.*3200.)],size,ax1,xystarts = xystarts)
       # ax1.plot(np.append(xbox*100./3200.,xbox[0]*100./3200.),np.append(ybox*100./3200.,ybox[0]*100./3200.),color='r')
       # plt.show()
    
    return SBdata